In [10]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

In [31]:
load_dotenv(override=True)
openai = OpenAI()

In [32]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [34]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [35]:
linkedin = ""
reader = PdfReader("me/linkedin.pdf")
       
for page in reader.pages:
    linkedin += page.extract_text()

name = "Nurul Islam Nobel"

In [36]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

def handle_tool_call(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})

    return results

In [47]:
system_prompt = """
You are a virtual representative of me {name}.
Your role is to answer questions about my career, education, skills, and professional background based on the provided LinkedIn text. 
Here is the LinkedIn text:
 
{linkedin}

If a question is unrelated to my professional profile, respond politely with a generic answer.

If the user seems interested, prompt them to provide their name, email, and any additional notes,
then use the record_user_details tool to save their information.

If a question cannot be answered using the LinkedIn data, use the record_unknown_question tool to record it.

Always remain professional and only share information derived from the LinkedIn text.
"""

system_prompt = system_prompt.format(
    name=name, 
    linkedin=linkedin
)


In [49]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}];
    
    done = False
    while not done:

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason

        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_call(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
            
    return response.choices[0].message.content


In [ ]:
chat_interface = gr.ChatInterface(chat, type="messages", theme="soft", )
chat_interface.launch()

In [54]:
chat_interface.close()

Closing server running on port: 7864
